Here we defined our own exposure estimation modell. Based on the input picture it predicts an amplification ratio.

Hyperparameters which can easily be changed.

In [66]:
HEIGHT = 512
WIDTH = 512
BATCHSIZE = 2

Here are all the imports.

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Embedding
from tensorflow.keras.optimizers import SGD

import os
import skimage.io as io
import skimage.transform as trans

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

import rawpy
import numpy as np

Ratio function with the appropiate exposure ratio. We use this as the groud truth.

In [69]:
def Ratio(in_path, truth_path):
    in_exposure = float(in_path[53:-5])
    gt_exposure = float(truth_path[52:-5])
    ratio = min(gt_exposure / in_exposure, 300)
    return ratio

for the model, we use a cnn with 3 conv. layers,dropout, and a flatten layer before the last dense with 1 output

In [70]:
model = Sequential() 
model.add(Conv2D(10, kernel_size=(3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, 3), strides= 2)) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(10, (5, 5), activation='relu', strides = 2)) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25))
model.add(Conv2D(10, (5, 5), activation='relu', strides = 2)) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten())
model.add(Dense(1))

#loss is mean squared error, optimizer is adam
model.compile(loss='mse', optimizer='adam')
model.summary()

Model = model

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 255, 255, 10)      280       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 127, 127, 10)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 62, 62, 10)        2510      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 31, 31, 10)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 31, 31, 10)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 14, 14, 10)        2510      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 7, 7, 10)         

load in the train dataset

In [71]:
trainPath = "/home/takats_balint1/deephf/data/Sony_train_list.txt"
replaceString = "/home/takats_balint1/deephf/data"

trainData = []
with open(trainPath) as File:
    for line in File:
        data, truth, _, _ = line.split()
        data = data.replace(".", replaceString, 1)
        truth = truth.replace(".", replaceString, 1)
        # This method is slow but we only have a 1000 lines so this quick and dirty algortihm will do just fine
        trainData.append([data, truth])
print (trainData[1])

['/home/takats_balint1/deephf/data/Sony/short/00001_00_0.1s.ARW', '/home/takats_balint1/deephf/data/Sony/long/00001_00_10s.ARW']


training the model: after reading the raw image in we resize it to match the input_size of the network.
Truth will be the reference/output for the network which we supposed to get after a succesful training

In [75]:
#Model = network()

trainPermutation = np.random.permutation(len(trainData[:16]))
data = [None]*BATCHSIZE
truth = [None]*BATCHSIZE
for i in range(0, len(trainPermutation), BATCHSIZE):
    for j in range(BATCHSIZE):
        
        ratio = Ratio(trainData[trainPermutation[i]][0], trainData[trainPermutation[i]][1])
        truth[j] = [ratio]
        raw = rawpy.imread(trainData[trainPermutation[i]][0])
        raw = raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        raw = np.float32(raw / 65535.0)
        raw = cv2.resize(raw, dsize=(HEIGHT, WIDTH), interpolation=cv2.INTER_CUBIC)

        data[j] = raw

    print("DATA LOADED")
    Model.train_on_batch(np.array(data), np.array(truth))


DATA LOADED
DATA LOADED
DATA LOADED
DATA LOADED
DATA LOADED
DATA LOADED
DATA LOADED
DATA LOADED
